In [6]:
import requests
import time
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [7]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/arenda-kvartir?per-page=50'

Добыча колличества ссылок

In [8]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

1402


Составляем массив ссылок

In [9]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 30/30 [00:13<00:00,  2.21it/s]

1402


Составление своей базы данных

In [10]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')
        print(link)

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Жилая недвижимость'
df['Подсегмент'] = 'Аренда квартир'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 1402/1402 [09:02<00:00,  2.58it/s]


Посмотрим на результаты вкрадце

In [11]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Комнат,Площадь,Этаж / этажность,...,Лоджия,Лифт,Мусоропровод,Наличие охраны,Торг уместен,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/arenda-kvartir/kvartira...,"Аренда квартиры-студии на ул. Родионова, д. 5Н...",14 000 руб./мес.,Bcя мeбeль и бытовaя техникa в нaличии и полнo...,2868971,Нижегородский,"ул. Родионова, д. 5",студия,32 / 18 / 7,2 / 5,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда квартир,Нижний Новгород,2022-12-21
1,https://www.gipernn.ru/arenda-kvartir/1-komnat...,"Аренда 1-комнатной квартиры на ул. Сурикова, д...",12 000 руб./мес.,Сдам однокомнатную квартиру в Приокском районе...,2818045,Приокский,"ул. Сурикова, д. 10",1 комната,"32 / 18,2 / 8",2 / 5,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда квартир,Нижний Новгород,2022-12-21
2,https://www.gipernn.ru/arenda-kvartir/1-komnat...,"Аренда 1-комнатной квартиры на б-р Южный, д. 1...",13 000 руб./мес.,Квартира сдается с хорошим ремонтом. С мебелью...,2828964,Автозаводский,"б-р Южный, д. 16",1 комната,37 / 15 / 13,7 / 17,...,есть,"пассажирский, грузовой",есть,NaN,NaN,gipernn,Жилая недвижимость,Аренда квартир,Нижний Новгород,2022-12-21
3,https://www.gipernn.ru/arenda-kvartir/1-komnat...,"Аренда 1-комнатной квартиры на ул. Окская, д. ...",15 000 руб./мес.,"Сдам квартиру с отличным ремонтом, делали под ...",2843334,Ленинский,"ул. Окская, д. 2",1 комната,40 / 17 / 9,11 / 23,...,NaN,"пассажирский, грузовой",есть,есть,NaN,gipernn,Жилая недвижимость,Аренда квартир,Нижний Новгород,2022-12-21
4,https://www.gipernn.ru/arenda-kvartir/1-komnat...,"Аренда 1-комнатной квартиры на ул. Победная, д...",13 000 руб./мес.,Сдам однокомнатную квартиру в Сормовском район...,2817808,Сормовский,"ул. Победная, д. 6",1 комната,32 / 18 / 8,7 / 14,...,NaN,пассажирский,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда квартир,Нижний Новгород,2022-12-21


Экспортируем таблицу

In [12]:
df.to_excel('flat-rent.xlsx', index=False)